# Model prediction maker

Model setup is as follows: we have dijet asymmetry data prepared, where the asymmetry AJ is defined as the difference between the two jets divided by the sum. Specifically,

$$A_{\mathrm{j}} = \frac{p_{\mathrm{T, 1}} - p_{\mathrm{T, 2}}}{p_{\mathrm{T, 1}} + p_{\mathrm{T, 2}}}$$

We will construct a model to describe the energy loss observed in the dijet asymmetry.  For this model, we consider back-to-back dijets.  Each jet can lose energy, and the lost energy is parameterized as

$$ \Delta p_{\mathrm{T}} / p_{\mathrm{T}} \sim A \times Gaus(1, B)$$

In addition to the energy loss contribution, we have extra "apparent" smearing on the AJ coming from the fact that we have other processes going on in the events (three jets etc).  It is parameterized as a Gaussian smearing on AJ with width C. So there are three total parameters: A, B, and C.

The measurement is done in two bins of centrality.  One in central event, where (A, B, C) are all relevant, and another one in very peripheral event, where only the parameter (C) is relevant.

The goal here in this notebook is to make the inputs needed for Bayesian inference to learn about A, B and C from the provided data

In [ ]:
from pathlib import Path

import numpy as np

folder = Path('input/XJGammaToy/')
folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# xj or Aj
#DataXMin        = 0.000
#DataXMax        = 1.000
#DataXBin        = 0.050
# xj gamma
DataXMin        = 0.000
DataXMax        = 2.000
DataXBin        = 0.125

DataNBin        = int((DataXMax - DataXMin) / DataXBin)

# how many design points do you want to generate?
NDesign         = 100

# What is the upper parameter range (one each for A, B, C)?
# The lower range for each parameter is 0 by construction.
# Hint: start with a large-range guess!  Then we can come back and reduce range
#ParameterRanges = [(0, 1.0), (0.0, 1.0), (-0.5, 1.5), (0.1, 1.0)]
ParameterRanges = [(0.025, 0.6), (0.025, 1.0), (0.025, 1.0), (0.1, 1.0)]

## The "prediction" function

Let's write a function, where we do the required smearing, make a histogram on the final AJ, and return the prediction

In [ ]:

import numpy.typing as npt


In [ ]:
def Predict(A, B, C, N = 100000):
    print("Running prediction with", A, B, C)
    
    Hist = np.zeros(DataNBin)
    
    for i in range(N):
        # Jet 1 and jet 2 PT (J1 and J2) after quenching.
        # Assuming initial energy is 100 GeV, and (delta PT / PT) ~ gaus(A, B), calculate the final energy
        # Jet PT = 100 GeV * (?)
        # Note that the initial energy cancels out in AJ
        # Useful function: np.random.normal(1, 2) gives you a random sampling with gaussian mean 1 and sigma 2
        J1 = 100 * (1 - A * np.random.normal(1, B))
        J2 = 100 * (1 - A * np.random.normal(1, B))
        # Calculate AJ from the PTs
        AJ = (J1 - J2) / (J1 + J2)
        # Adding extra gaussian smearing from parameter C
        AJ = AJ + np.random.normal(0, C)
        # AJ is defined to be leading - subleading -> positive!
        AJ = np.abs(AJ)

        # put things into bins
        Bin = int((AJ - DataXMin) / DataXBin)
        if Bin < 0:   # underflow
            Bin = 0
        if Bin >= DataNBin:   # overflow
            continue
            # Bin = DataNBin - 1
        
        Hist[Bin] = Hist[Bin] + 1
        
    return Hist / N

def predict_xjgamma(A: float, B: float, C: float, D: float, E: float, F: float, N: int = 100000) -> npt.NDArray[np.float64]:
    print(f"Running xj_gamma prediction with {A}, {B}, {C}, {D}")
    
    hist = np.zeros(DataNBin)
    
    for i in range(N):
        # Jet 1 and jet 2 PT (J1 and J2) after quenching.
        # Assuming initial energy is 100 GeV, and (delta PT / PT) ~ gaus(A, B), calculate the final energy
        # Jet PT = 100 GeV * (?)
        # Note that the initial energy cancels out in AJ
        # Useful function: np.random.normal(1, 2) gives you a random sampling with gaussian mean 1 and sigma 2
        J1 = 100 * (1 - A * np.random.normal(0.1, B))
        #J2 = 100 * (1 - np.random.normal(A, B))
        J2 = 100 * (1 - C * np.random.normal(0.01, D))
        # Calculate AJ from the PTs
        XJ = J1 / J2
        # Adding extra gaussian smearing from parameter C
        XJ = XJ + np.random.normal(E, F)
        # AJ is defined to be leading - subleading -> positive!
        #AJ = np.abs(AJ)

        # put things into bins
        Bin = int((XJ - DataXMin) / DataXBin)
        if Bin < 0:   # underflow
            Bin = 0
        if Bin >= DataNBin:   # overflow
            continue
            # Bin = DataNBin - 1
        
        hist[Bin] = hist[Bin] + 1
        
    return hist / N

def predict_xj(A: float, B: float, C: float, D: float, N: int = 100000) -> npt.NDArray[np.float64]:
    print(f"Running xj prediction with {A}, {B}, {C}, {D}")
    
    hist = np.zeros(DataNBin)
    
    for i in range(N):
        # Jet 1 and jet 2 PT (J1 and J2) after quenching.
        # Assuming initial energy is 100 GeV, and (delta PT / PT) ~ gaus(A, B), calculate the final energy
        # Jet PT = 100 GeV * (?)
        # Note that the initial energy cancels out in AJ
        # Useful function: np.random.normal(1, 2) gives you a random sampling with gaussian mean 1 and sigma 2
        #J1 = 100 * (1 - A * np.random.normal(1, B))
        #J2 = 100 * (1 - A * np.random.normal(1, B))
        J1 = 100 * (1 - np.abs(np.random.normal(A, B)))
        #J2 = 100 * (1 - np.random.normal(A, B))
        J2 = 100 * (1 - np.abs(np.random.normal(A, B)))
        if J1 > J2:
            J1, J2 = J2, J1
        # Calculate XJ from the PTs
        XJ = J1 / J2
        # Adding extra gaussian smearing from parameter C
        #XJ = XJ + np.random.normal(0.7, D)
        XJ = XJ + np.random.normal(0.8, D)
        # AJ is defined to be leading - subleading -> positive!
        # OVERFLOW IS THE ISSUE! Too broad!
        if XJ > 2:
            XJ = 2
        if XJ > 1:
            print(XJ)
            XJ = XJ - 2 * (XJ - 1)
            print(XJ)
        #AJ = np.abs(AJ)

        # put things into bins
        Bin = int((XJ - DataXMin) / DataXBin)
        #if Bin == 0:
        #    print(J1, J2)
        if Bin < 0:   # underflow
            Bin = 0
        if Bin >= DataNBin:   # overflow
            continue
            # Bin = DataNBin - 1
        
        hist[Bin] = hist[Bin] + 1
        
    return hist / N


def predict_xjgamma_smeared(A: float, B: float, C: float, D: float, N: int = 100000) -> npt.NDArray[np.float64]:
    print(f"Running xj_gamma smeared prediction with {A}, {B}, {C}, {D}")
    
    bins = np.linspace(DataXMin, DataXMax, int((DataXMax - DataXMin) / DataXBin) + 1)
    hists = []
    #hist = np.zeros(DataNBin)
    
    xj_values = []
    for i in range(N):
        # Jet 1 and jet 2 PT (J1 and J2) after quenching.
        # Assuming initial energy is 100 GeV, and (delta PT / PT) ~ gaus(A, B), calculate the final energy
        # Jet PT = 100 GeV * (?)
        # Note that the initial energy cancels out in AJ
        # Useful function: np.random.normal(1, 2) gives you a random sampling with gaussian mean 1 and sigma 2
        #J1 = 100 * (1 - np.random.normal(A, B))
        J1 = 100 * (1 - np.random.normal(A, B))
        #J2 = 100 * (1 - np.random.normal(A, B))
        J2 = 100
        # Calculate AJ from the PTs
        XJ = J1 / J2
        # Adding extra gaussian smearing from parameter C
        smear = np.random.normal(C, D)
        xj_original = XJ
        XJ = XJ + smear
        # XJ must be positive definition. If the smearing is too large, set to 0
        if XJ < 0:
            XJ = 0
        # AJ is defined to be leading - subleading -> positive!
        #AJ = np.abs(AJ)

        h, _ = np.histogram(a=XJ, bins=bins)
        print(f"{h}")
        hists.append(h / h.sum())

        ## put things into bins
        #Bin = int((XJ - DataXMin) / DataXBin)
        #if Bin == 0:
        #    pass
        #    #print(f"{J1=}, {J2=}, {XJ=}, {smear=}, {xj_original=}")
        #if Bin < 0:   # underflow
        #    Bin = 0
        #if Bin >= DataNBin:   # overflow
        #    continue
        #    # Bin = DataNBin - 1
        
        #hist[Bin] = hist[Bin] + 1
        
    #return hist / N
    return hists[0] if len(hists) == 1 else hists

In [ ]:
def predict_xjgamma_smeared_full(A: float, B: float, C: float, D: float, N: int = 100000) -> npt.NDArray[np.float64]:
    print(f"Running xj_gamma smeared prediction with {A}, {B}, {C}, {D}")
    
    # Jet 1 and jet 2 PT (J1 and J2) after quenching.
    # Assuming initial energy is 100 GeV, and (delta PT / PT) ~ gaus(A, B), calculate the final energy
    # Jet PT = 100 GeV * (?)
    # Note that the initial energy cancels out in AJ
    # Useful function: np.random.normal(1, 2) gives you a random sampling with gaussian mean 1 and sigma 2
    #J1 = 100 * (1 - np.random.normal(A, B))
    J1 = 100 * (1 - np.random.normal(A, B, size=(N, len(A))))
    #J2 = 100 * (1 - np.random.normal(A, B))
    J2 = 100
    # Calculate AJ from the PTs
    XJ = J1 / J2
    # Adding extra gaussian smearing from parameter C
    smear = np.random.normal(C, D, size=(N, len(A)))
    xj_original = XJ
    XJ = XJ + smear
    # XJ must be positive definition. If the smearing is too large, set to 0
    _mask = (XJ < 0)
    XJ[_mask] = 0
    # AJ is defined to be leading - subleading -> positive!
    #AJ = np.abs(AJ)

    # put things into bins
    bins = np.linspace(DataXMin, DataXMax, int((DataXMax - DataXMin) / DataXBin) + 1)
    #Bin = int((XJ - DataXMin) / DataXBin)
    #if Bin == 0:
    #    pass
    #    #print(f"{J1=}, {J2=}, {XJ=}, {smear=}, {xj_original=}")
    #if Bin < 0:   # underflow
    #    Bin = 0
    #if Bin >= DataNBin:   # overflow
    #    continue
    #    # Bin = DataNBin - 1
    
    #hist[Bin] = hist[Bin] + 1

    #h, _ = np.histogram2d(x=XJ, y=bins=bins)
    #h, _ = np.histogram(XJ, y=bins=bins)
   
    #return hist / N
    #return h / h.sum()

In [ ]:
def predict_xjgamma_smeared_fix(A: float, B: float, C: float, D: float, n_samples: int = 100000) -> npt.NDArray[np.float64]:
    print(f"Running xj_gamma smeared prediction with {A}, {B}, {C}, {D}")
    
    bins = np.linspace(DataXMin, DataXMax, int((DataXMax - DataXMin) / DataXBin) + 1)
    #hists = []
    #hist = np.zeros(DataNBin)

    # Jet 1 and jet 2 PT (J1 and J2) after quenching.
    # Assuming initial energy is 100 GeV, and (delta PT / PT) ~ gaus(A, B), calculate the final energy
    # Jet PT = 100 GeV * (?)
    # Note that the initial energy cancels out in AJ
    # Useful function: np.random.normal(1, 2) gives you a random sampling with gaussian mean 1 and sigma 2
    #J1 = 100 * (1 - np.random.normal(A, B))
    J1 = 100 * (1 - np.random.normal(A, B, size=n_samples))
    #J2 = 100 * (1 - np.random.normal(A, B))
    J2 = 100
    # Calculate AJ from the PTs
    XJ = J1 / J2
    # Adding extra gaussian smearing from parameter C
    smear = np.random.normal(C, D, size=n_samples)
    xj_original = XJ
    #XJ = XJ - smear
    XJ = XJ + smear
    # XJ must be positive definition. If the smearing is too large, we leave them below 0
    # and they are subsequently ignored in the histogramming
    #_mask = (XJ < 0)
    #XJ[_mask] = 0
    # AJ is defined to be leading - subleading -> positive!
    #AJ = np.abs(AJ)

    #print(f"{smear=}, {xj_original=}")
    #return XJ
    h, _ = np.histogram(a=XJ, bins=bins)
    #print(f"{h=}")
    #hists.append(h / h.sum())

    ## put things into bins
    #Bin = int((XJ - DataXMin) / DataXBin)
    #if Bin == 0:
    #    pass
    #    #print(f"{J1=}, {J2=}, {XJ=}, {smear=}, {xj_original=}")
    #if Bin < 0:   # underflow
    #    Bin = 0
    #if Bin >= DataNBin:   # overflow
    #    continue
    #    # Bin = DataNBin - 1
    
    #hist[Bin] = hist[Bin] + 1
        
    #return hist / N
    #return hists[0] if len(hists) == 1 else hists
    return h / h.sum()

### Test the prediction (cross check for yourself)

In [ ]:
# Test predicting one point - to see if the output makes sense or not
# Once you are happy, we move on!
#example_prediction = predict_xjgamma(1, 0.25, 0, 0, 0, 0.3)
#example_prediction = predict_xj(0.1, 0.3, 1, 1.5)
example_prediction = predict_xjgamma_smeared_fix(1, 0, 1.71608035669855451, 0.142)
example_prediction

In [ ]:
example_prediction.min()

In [ ]:
# Alternatively (or in addition), plot the AJ distribution for our single point
import matplotlib.pyplot as plt
#example_prediction = predict_xj(0.3, 0.3, 0, 0.3)
#example_prediction = predict_xjgamma_smeared(0.2, 0.6, 0.2, 0.3)
example_prediction = predict_xjgamma_smeared_fix(0.05, 0.3, -0.1, 0.3)
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(np.arange(DataXMin, DataXMax, DataXBin) + (DataXBin / 2), example_prediction, marker="o", linestyle="")

In [ ]:
# Test

In [ ]:
%pip install ipywidgets

In [ ]:
%pip install ipympl

In [ ]:
_x = np.arange(DataXMin, DataXMax, DataXBin) + (DataXBin / 2)

In [ ]:
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(_x, predict_xj(A=0.1, B=0.3, C=1, D=0.3))

x = np.array([1,2,3,4,5])

def update(A = 1, B = 0, C = 0):
    line.set_ydata(f(x,A,B,C))
    fig.canvas.draw_idle()
    
interact(update, A = (0,1,0.1), B = (0,1,0.1), C = (1,1,0.1), D = (1,1,0.1))

In [ ]:
from typing import Union
import numpy.typing as npt
_x = np.linspace(0, 2, 20)

def gaussian(
    x: Union[npt.NDArray[np.float64], float], mean: float, sigma: float
) -> Union[npt.NDArray[np.float64], float]:
     r"""Normalized gaussian.

     .. math::

         f = 1 / \sqrt{2 * \pi * \sigma^{2}} * \exp{-\frac{(x - \mu)^{2}}{(2 * \sigma^{2}}}

     Args:
         x: Value(s) where the gaussian should be evaluated.
         mean: Mean of the gaussian distribution.
         sigma: Width of the gaussian distribution.
     Returns:
         Calculated gaussian value(s).
     """
     return 1.0 / np.sqrt(2 * np.pi * np.square(sigma)) * np.exp(-1.0 / 2.0 * np.square((x - mean) / sigma))  # type: ignore

def extended_gaussian(
    x: Union[npt.NDArray[np.float64], float], mean: float, sigma: float, amplitude: float
) -> Union[npt.NDArray[np.float64], float]:
    r"""Extended gaussian.

    .. math::

        f = A / \sqrt{2 * \pi * \sigma^{2}} * \exp{-\frac{(x - \mu)^{2}}{(2 * \sigma^{2}}}

    Args:
        x: Value(s) where the gaussian should be evaluated.
        mean: Mean of the gaussian distribution.
        sigma: Width of the gaussian distribution.
        amplitude: Amplitude of the gaussian.
    Returns:
        Calculated gaussian value(s).
    """
    return amplitude / np.sqrt(2 * np.pi * np.square(sigma)) * np.exp(-1.0 / 2.0 * np.square((x - mean) / sigma))  # type: ignore

def test_func(x: npt.NDArray[np.float64], a: float, b: float, c: float, d: float, e: float) -> float:
    return e*(1-gaussian(x, a, b)) - gaussian(x, c, d)

In [ ]:
_x = np.arange(DataXMin, DataXMax, DataXBin) + (DataXBin / 2)
fig, ax = plt.subplots()
ax.plot(_x, extended_gaussian(_x, 1, 0.1, 1) + extended_gaussian(_x, 1, 0.8, 3), marker="o", alpha=0.5)
ax.plot(_x, extended_gaussian(_x, 1, 0.1, 1), marker="o", alpha=0.5)
ax.plot(_x, extended_gaussian(_x, 1, 0.8, 3), marker="o", alpha=0.5)
#ax.plot(_x, extended_gaussian(_x, 1, 1, 5), marker="o", alpha=0.5)
ax.plot(_x, 5*gaussian(_x, 1, 1), marker="s", alpha=0.5)

## Making the design points

Let's start with a very simple random array :D

In reality we would use something more complicated to distribute the points better, but let's start simple.  Fancy stuff is just a better way to achieve the same purpose.

In [ ]:
#Design = np.random.rand(NDesign, 3) * ParameterRanges
design = np.array([
    np.random.uniform(low=p[0], high=p[1], size=NDesign)
    for p in ParameterRanges
]).T

In [ ]:
design[:, 0].tolist()

In [ ]:
def f(x, y):
    return x + y

my_x = np.random.rand(10)
my_y = np.random.rand(10)
np.allclose(f(my_x, my_y), my_x + my_y)

In [ ]:
Y1 = np.array([
    #np.vectorize(predict_xjgamma_smeared_fix)(A=design[0, :], B=design[1, :], C=design[2, :], D=design[3, :]) for d in design
    predict_xjgamma_smeared_fix(A=d[0], B=d[1], C=d[2], D=d[3]) for d in design.T
])

In [ ]:
Y1[0]

In [ ]:
predict_xjgamma_smeared_fix(A=design[-1, 0], B=design[-1, 1], C=design[-1, 2], D=design[-1, 3])

## Preparing the model predictions

Let's loop over the design points, and call the predict function we just wrote to make a big table!

This step takes a while, like a few minutes.  Please be patient.

In [ ]:
# Generate prediction for "central" data
#Y1 = [predict_xjgamma_smeared(i[0], i[1], i[2]) for i in design]
Y1 = np.array([
    predict_xjgamma_smeared_fix(A=d[0], B=d[1], C=d[2], D=d[3]) for d in design
])
# Generate prediction for "peripheral" data.  Note here A and B are irrelevant.  So we set them to 0
#Y2 = [predict_xjgamma_smeared(0, 0, i[2]) for i in design]
Y2 = np.array([
    # A = 1 so that the XJ cancels out to 0, and then it's just the second term that dominates
    predict_xjgamma_smeared_fix(A=1, B=0.02, C=d[2], D=d[3]) for d in design
])

In [ ]:
Y2 = np.array([
    # A = 1 so that the XJ cancels out to 0, and then it's just the second term that dominates
    predict_xjgamma_smeared_fix(A=1, B=0.02, C=d[2], D=d[3]) for d in design[26:30]
])
#Y2

## Write everything out

In [ ]:
with open(folder / 'Prediction_Selection1.dat', 'w') as f:
    f.write('# Version 1.0\n')
    f.write('# Data Data_Selection1.dat\n')
    f.write('# Design Design.dat\n')
    np.savetxt(f, np.transpose(Y1))

In [ ]:
with open(folder / 'Prediction_Selection2.dat', 'w') as f:
    f.write('# Version 1.0\n')
    f.write('# Data Data_Selection2.dat\n')
    f.write('# Design Design.dat\n')
    np.savetxt(f, np.transpose(Y2))

In [ ]:
with open(folder / 'Design.dat', 'w') as f:
    f.write('# Version 1.0\n')
    f.write(f'# Parameter {" ".join("A B C D E F G H I J K L".split()[:design.shape[1]])}\n')
    np.savetxt(f, design)

## Making fake data

In [ ]:
Truth = [0.50, 0.25, 0.10]

DataY1 = Predict(Truth[0], Truth[1], Truth[2], N = 100000)
DataY2 = Predict(0, 0, Truth[2], N = 100000)

In [ ]:
XMin = np.array(range(0, DataNBin)) * DataXBin
XMax = np.array(range(1, DataNBin + 1)) * DataXBin

Stat = 0.001
Sys = 0.010

Data1 = np.zeros((DataNBin, 7))
Data2 = np.zeros((DataNBin, 7))

Data1[:,0] = XMin
Data1[:,1] = XMax
Data1[:,2] = DataY1
Data1[:,3] = Stat
Data1[:,4] = Stat
Data1[:,5] = Sys
Data1[:,6] = Sys

Data2[:,0] = XMin
Data2[:,1] = XMax
Data2[:,2] = DataY2
Data2[:,3] = Stat
Data2[:,4] = Stat
Data2[:,5] = Sys
Data2[:,6] = Sys

In [ ]:
with open(folder / 'Data_Selection1.dat', 'w') as f:
    f.write('# Version 1.0\n')
    f.write('# DOI None\n')
    f.write('# Source None\n')
    f.write('# Experiment JetScape\n')
    f.write('# System PbPb5020\n')
    f.write('# Centrality 0to10\n')
    f.write('# XY AJ DNDAJ\n')
    f.write('# Label xmin xmax y stat,low stat,high sys,low sys,high\n')
    np.savetxt(f, Data1)
    
with open(folder / 'Data_Selection2.dat', 'w') as f:
    f.write('# Version 1.0\n')
    f.write('# DOI None\n')
    f.write('# Source None\n')
    f.write('# Experiment JetScape\n')
    f.write('# System PbPb5020\n')
    f.write('# Centrality 70to90\n')
    f.write('# XY AJ DNDAJ\n')
    f.write('# Label xmin xmax y stat,low stat,high sys,low sys,high\n')
    np.savetxt(f, Data2)